In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Data loading and processing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

# Defining the model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Creating an instance of the model and moving it to GPU if available
model = Model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Defining the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

# Training the model
for epoch in range(6):  # fixed to 6 epochs
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_acc = correct_train / total_train
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss}, Train Accuracy: {train_acc}')

# Test the model on the test set and print the accuracy
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs_test = model(images)
        _, predicted_test = torch.max(outputs_test.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted_test == labels).sum().item()

test_acc = correct_test / total_test
print(f'Test Accuracy: {test_acc}')

100%|██████████| 170498071/170498071 [00:03<00:00, 49226550.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Train Loss: 1.3548841463855956, Train Accuracy: 0.5141
Epoch 2, Train Loss: 0.9441920802788482, Train Accuracy: 0.66822
Epoch 3, Train Loss: 0.770500762277281, Train Accuracy: 0.72832
Epoch 4, Train Loss: 0.6595978034530323, Train Accuracy: 0.76622
Epoch 5, Train Loss: 0.5605788915796457, Train Accuracy: 0.80258
Epoch 6, Train Loss: 0.47405354767294167, Train Accuracy: 0.83266
Test Accuracy: 0.7331
